#### Swin Polyp classification: Polyp or No polyp

In [1]:
import torchvision

In [2]:
from torchvision.models import swin_b

In [3]:
import torch

In [4]:
import numpy as np
import os
import pandas as pd

In [5]:
from torchvision import datasets, transforms

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
# data_dir = '/kaggle/input/data/'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [8]:
ABS_PATH  = os.path.abspath('')

In [9]:
dataset_path = os.path.join(ABS_PATH,"TrainingSet_NewGT/polyp_classification")

In [10]:
dataset = datasets.ImageFolder(dataset_path, transform=transform)

In [11]:
len(dataset)

2979

In [12]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

In [13]:
import torch.nn as nn
import torch.optim as optim

In [14]:
# from torchvision.models import swin_t
# Use a pre-trained imagenet
#model = swin_t(weights='DEFAULT')
# Use no weights
model = swin_b()

# Update the fully connected layer based on the number of classes in the dataset
#model.fc = torch.nn.Linear(model.fc.in_features, len(ds_train.labels.info.class_names))
model.head = torch.nn.Linear(out_features=1,in_features=1024)

model.to(device)

print(model)
# Specity the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

SwinTransformer(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (1): Sequential(
      (0): SwinTransformerBlock(
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttention(
          (qkv): Linear(in_features=128, out_features=384, bias=True)
          (proj): Linear(in_features=128, out_features=128, bias=True)
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=512, out_features=128, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (1): SwinTransformerBlock

In [15]:
from sklearn.metrics import roc_auc_score

In [16]:
from torchmetrics import AUROC
auroc = AUROC(task='binary')

In [18]:
from tqdm import tqdm

#train
model.train()
num_epochs = 5
for epoch in range(num_epochs):
    
    running_loss = 0.0
    running_auc=0.0
    running_auroc = 0.0
    for images, labels in tqdm(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        labels = labels.unsqueeze(1)
        labels = labels.float()
        loss = criterion(outputs, labels)
        auroc_score = auroc(outputs,labels)
        running_auroc += auroc_score * images.size(0)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        # labels = labels.detach().cpu()
        # outputs = outputs.detach().cpu()
        # auc = roc_auc_score(labels,outputs)
        # running_auc += auc * images.size(0)
        
    # auc_score = running_auc/len(dataset)
    auroc_score = running_auroc/len(dataset)
    epoch_loss = running_loss / len(dataset)
    # print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss))
    print('Epoch [{}/{}], Loss: {:.4f}, AUROC_Score {:.4f}'.format(epoch+1, num_epochs, epoch_loss, auroc_score))

100%|██████████| 298/298 [04:08<00:00,  1.20it/s]


Epoch [1/5], Loss: 0.1669, AUROC_Score


100%|██████████| 298/298 [02:53<00:00,  1.72it/s]


Epoch [2/5], Loss: 0.1399, AUROC_Score


100%|██████████| 298/298 [02:58<00:00,  1.67it/s]


Epoch [3/5], Loss: 0.1195, AUROC_Score


 27%|██▋       | 79/298 [00:48<02:18,  1.58it/s]d:\Anaconda\envs\xai\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
100%|██████████| 298/298 [03:05<00:00,  1.61it/s]


Epoch [4/5], Loss: 0.0975, AUROC_Score


 14%|█▍        | 41/298 [00:26<02:38,  1.62it/s]d:\Anaconda\envs\xai\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
100%|██████████| 298/298 [03:11<00:00,  1.55it/s]

Epoch [5/5], Loss: 0.0641, AUROC_Score
